In [28]:
import pandas as pd
import requests, json
import numpy as np
import folium
import re
import json
import datetime
import xml.etree.ElementTree as Xml

In [29]:
minlon, minlat, maxlon, maxlat = 77.7519486464,12.9833797192,77.7528560462,12.9842016832

minlat=str(minlat)
minlon=str(minlon)
maxlat=str(maxlat)
maxlon=str(maxlon)

In [30]:
hermap_filename = 'data_comparision/heremap.json'
sumo_filename = 'data_comparision/original_sumo.net.xml'

In [31]:
with open(hermap_filename) as f:
    heremap_json = json.load(f)

In [32]:
e = Xml.parse(sumo_filename).getroot()
for i in e:
    print(i)

<Element 'location' at 0x7f899a0e18b8>
<Element 'type' at 0x7f899a0e1098>
<Element 'type' at 0x7f899a0e19a8>
<Element 'type' at 0x7f899a0e1a98>
<Element 'type' at 0x7f899a0e1ae8>
<Element 'type' at 0x7f899a0e1b38>
<Element 'type' at 0x7f899a0e1b88>
<Element 'type' at 0x7f899a0e1bd8>
<Element 'type' at 0x7f899a0e1c28>
<Element 'type' at 0x7f899a0e1c78>
<Element 'type' at 0x7f899a0e1cc8>
<Element 'type' at 0x7f899a0e1d18>
<Element 'type' at 0x7f899a0e1d68>
<Element 'type' at 0x7f899a0e1db8>
<Element 'type' at 0x7f899a0e1e08>
<Element 'type' at 0x7f899a0e1e58>
<Element 'type' at 0x7f899a0e1ea8>
<Element 'type' at 0x7f899a0e1f48>
<Element 'type' at 0x7f899a0e1f98>
<Element 'type' at 0x7f899a070048>
<Element 'type' at 0x7f899a070098>
<Element 'type' at 0x7f899a0700e8>
<Element 'type' at 0x7f899a070138>
<Element 'type' at 0x7f899a070188>
<Element 'type' at 0x7f899a0701d8>
<Element 'type' at 0x7f899a070228>
<Element 'type' at 0x7f899a070278>
<Element 'type' at 0x7f899a0702c8>
<Element 'type' 

In [23]:
def create_data_1(dictionary):
        
    RWS=dictionary['RWS']
    l=[]
    for rw in RWS:
        EBU_COUNTRY_CODE=rw['EBU_COUNTRY_CODE']
        EXTENDED_COUNTRY_CODE=rw['EXTENDED_COUNTRY_CODE']
        UNITS=rw['UNITS']
        for elem in rw['RW']:
            DE=elem['DE']
            LI=elem['LI']
            PBT=elem['PBT']
            mid=elem['mid']
            FIS=elem['FIS']
            for FI in FIS:
                elem_1=FI['FI']
                for elem_2 in elem_1:
                    LE=elem_2['TMC']['LE']
                    PC=elem_2['TMC']['PC']
                    QD=elem_2['TMC']['QD']
                    SHP=elem_2['SHP']
                    CF=elem_2['CF']
                    for elem_3 in CF:
                        FF=elem_3['FF']
                        CN=elem_3['CN']                    
                        JF=elem_3['JF']
                        SP=elem_3['SP']
                        TY=elem_3['TY']
                        l.append([EBU_COUNTRY_CODE,EXTENDED_COUNTRY_CODE,UNITS,DE,LI,PBT,mid,LE,PC,QD,SHP,FF,CN,JF,SP,TY])
    clms=['EBU_COUNTRY_CODE','EXTENDED_COUNTRY_CODE','UNITS','DE','LI','PBT','mid','LE','PC','QD','SHP','FF','CN','JF','SP','TY']                    
    df=pd.DataFrame(l,columns=clms)
    df['PBT']=datetime.datetime.now()
    #print(json.dumps(rws_list, indent=2))
#     if path_to_save:
#         if os.path.isfile(path_to_save):
#             df.to_csv(path_to_save,mode='a+',header=False,index=False)
#         else:
#             df.to_csv(path_to_save,mode='w+',index=False)
#     else:
    df['SHP'] = df['SHP'].astype(str)
    return df  

def traffic_map_across_city_historical(traffic_data):
    
    traffic_hour= traffic_data
    df_jaipur=traffic_data
    df_ori=df_jaipur
    list_shp = df_jaipur['SHP'].unique()
    list_mean = []
    start_point=[]
    end_point=[]
    for s in list_shp:
        tmp = re.findall("[+-]?\d*\.\d+", s)
        #print(tmp)
        tmp=[float(i) for i in tmp]
        tmp2 = []
        it = iter(tmp)
        for z in it:
            tmp2.append([z, next(it)])
        a = np.array(tmp2)
        list_mean.append(list(np.mean(a, axis=0)))
        start_point.append(tmp2[0])
        end_point.append(tmp2[-1])
    df_t = pd.DataFrame()
    df_t['SHP'] = list_shp
    df_t['mean'] = list_mean
    df_final =df_jaipur.merge(df_t,on='SHP')
    df_final['latitude'] = np.array(df_final['mean'].values.tolist())[:,0]
    df_final['longitude'] = np.array(df_final['mean'].values.tolist())[:,1]
    hh=pd.pivot_table(df_final,index=['SHP','PC','DE'],values='JF',aggfunc='median').reset_index()
    da1=hh.sort_values('JF',ascending=False)
    #da2=da1[da1['JF']>5]
    da2=da1

    data_2=da2
    data_2=data_2
    import branca.colormap as cm

    colormap = cm.LinearColormap(['green','yellow','red'],vmin=0,vmax=10)
    colormap.caption="Jamfactor_levels"
#     lat_long=df[['latitude','longitude']].values[0].tolist()
    x=folium.Map(location=[12.9380131671,77.6319900345],zoom_start=10)
    # x

    for i in data_2.index.tolist():
        color_grad_val=data_2['JF'][i]
        tmp2=[]
        #tmp=re.findall("\d*\.\d+", data_2['SHP'][i])
        tmp = re.findall("[+-]?\d*\.\d+", data_2['SHP'][i])
        #print(tmp)
        tmp=[float(i) for i in tmp]

        it = iter(tmp)
        for z in it:
            tmp2.append([z, next(it)])
        y=folium.PolyLine(locations=tmp2,color=colormap.rgb_hex_str(color_grad_val),opacity=0.75,weight=7)
        x.add_children(y)
        
    
    df= df_final[['latitude','longitude','PC']].drop_duplicates()

    l = df[['latitude','longitude']].values.tolist()

    #map_osm = folium.Map(location=[38.946241233265095, -77.04115322627564],tiles='CartoDB Positron',
    #                     zoom_start=12)
    for i,k in zip(l,df['PC']):

        marker = folium.Marker(i,popup=str(k),icon=folium.Icon())

        x.add_child(marker)

    

    x
    display(x)

In [25]:
heremap_data = create_data_1(heremap_json)
traffic_map_across_city_historical(heremap_data)

/home/utkarsh/.virtualenvs/Traffic_Simulation/lib/python3.5/site-packages/ipykernel_launcher.py:97: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.
